<a href="https://colab.research.google.com/github/GilbertoQ/Daily_FX_ML/blob/master/dailyFX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import pandas
import numpy as np
import random

In [0]:
t = pandas.read_csv("/content/drive/My Drive/Colab Notebooks/Financial-CNN/Data/Daily_FX/all_daily_FX_rates.csv",sep=",", header=0 )
t['DATE'] =  pandas.to_datetime(t['DATE'])
t=t.fillna(0)
print(t)
#t = t.set_index('DATE')

In [0]:
#Starting Index for training data
TRAIN_START = 30

#Ending index for training data: non-inclusive
TRAIN_END = int(len(t)*.85)

#Total training Data
STEPS_PER_EPOCH = TRAIN_END-TRAIN_START
print(STEPS_PER_EPOCH)
NUMBER_OF_EPOCHS = 5

#number of previous days to put in each sample
HISTORY=27

CLASSES = 5

SAMPLE_HEIGHT = SAMPLE_WIDTH = 26
SAMPLE_DEPTH=1

CLASS_WEIGHTS={0:1.5,
               1:1.5,
               2:1.,
               3:1.5,
               4:1.5}

10782


In [0]:
#This cell gets statistics of the data
cols=['DEXBZUS','DEXCAUS' ,'DEXCHUS', 'DEXDNUS' ,'DEXHKUS' ,'DEXINUS' ,'DEXJPUS'
 ,'DEXKOUS' ,'DEXMAUS', 'DEXMXUS', 'DEXNOUS' ,'DEXSDUS' ,'DEXSFUS', 'DEXSIUS'
 ,'DEXSLUS' ,'DEXSZUS' ,'DEXTAUS', 'DEXTHUS' ,'DEXUSAL', 'DEXUSEU' ,'DEXUSNZ'
 ,'DEXUSUK' ,'DEXVZUS' ,'DTWEXB' ,'DTWEXM' ,'DTWEXO']
#get average change and max
for j in range(len(cols)):
  avg_up=(0,0)
  avg_down=(0,0)
  a_u=[]
  a_d=[]
  temp=j
  maxs=(0,0)
  mins= (100,0)
  for i in range(1,len(t[cols[temp]])):
    if t[cols[temp]][i-1]!=0:
      temp2=t[cols[temp]][i]-t[cols[temp]][i-1]
      if temp2>maxs[0]:
        maxs=(temp2,i)
      if temp2<mins[0]:
        mins=(temp2,i)
      if temp2 > 0:
        avg_up=(avg_up[0]+temp2,avg_up[1]+1)
        a_u.append(temp2)
      elif temp2<0:
        avg_down=(avg_down[0]+temp2,avg_down[1]+1)
        a_d.append(temp2)
  a_u=pandas.DataFrame(a_u)
  a_d=pandas.DataFrame(a_d)
  print(cols[temp]+" "+str(a_u.describe())+" "+str(a_d.describe()))

In [0]:
#used pandas describe for these numbers
def categorize_USAL_delta(delta):
  if delta >= 0.004500:
    return 4
  if delta >= 0.002400:
    return 3
  if delta <= -0.005100:
    return 0
  if delta <= -0.002200:
    return 1
  return 2

In [0]:
def change_to_deltas(index,mat):
  cols=['DEXBZUS','DEXCAUS' ,'DEXCHUS', 'DEXDNUS' ,'DEXHKUS' ,'DEXINUS' ,'DEXJPUS'
 ,'DEXKOUS' ,'DEXMAUS', 'DEXMXUS', 'DEXNOUS' ,'DEXSDUS' ,'DEXSFUS', 'DEXSIUS'
 ,'DEXSLUS' ,'DEXSZUS' ,'DEXTAUS', 'DEXTHUS' ,'DEXUSAL', 'DEXUSEU' ,'DEXUSNZ'
 ,'DEXUSUK' ,'DEXVZUS' ,'DTWEXB' ,'DTWEXM' ,'DTWEXO']
  for i in cols:
    for j in range(index-1,index-HISTORY,-1):
      #print(j)
      nex=mat[i][j-1]
      curr=mat[i][j]
      if nex != 0 and curr!=0:
        #get deltas
        mat[i][j]=curr-nex
      elif nex == 0 and curr!=0:
        #make mat[i][j]=0
         mat[i][j]=0
  mat = mat.drop([index-HISTORY,index])
  #Min-max normalization
  #[mat[col].update((mat[col] - mat[col].min()) / (mat[col].max() - mat[col].min())) for col in mat.columns]
  #Z-score normalization
  [mat[col].update((mat[col] - mat[col].mean()) / mat[col].std()) for col in mat.columns]
  return mat
  #once the deltas are aqui

In [0]:
def gen_mat_with_aws(index,data):
  if data.iloc[index+1][0] == 0:
    return 0,0 
  mat=data.iloc[index-HISTORY:index+1]
  mat=mat.copy()
  aws=mat['DEXUSAL'][index]-mat['DEXUSAL'][index-1]
  del mat['DATE']
  
  mat=change_to_deltas(index,mat)
  
  #print(aws)
  aws = categorize_USAL_delta(aws)
  #print(aws)
  #make columns into rows for easier use
  mat=mat.transpose()
  #make it a nparray
  mat=mat.values
  mat = mat.astype('float32')
  mat = mat.reshape((-1,SAMPLE_HEIGHT * SAMPLE_WIDTH,1))
  #print(mat)
  return mat,aws
  
  

In [0]:
#get random list of numbers from 30:len(data)-2000, this ensures valid data and testing set
#get valid matrix: the first and last columns and are only used to refrence coorect changes in price
#change the columns to rows for eaiser use
#change data to deltas
#print(t)
#range non-inclusive: hence len(t)-2000 not in rand_list

#print(len(rand_list))

def fit_generator():
  print("start")
  while True:
    rand_list = random.sample(range(TRAIN_START,TRAIN_END),STEPS_PER_EPOCH)
    for i,n in enumerate(rand_list):
      mat,aws = gen_mat_with_aws(n,t)
      #one_hot = [0 for _ in range(CLASSES) ]
      #one_hot[aws]=1
      yield mat,[aws]


In [0]:
#ML Sections
#!pip install tensorflow-gpu==2.0.0
import tensorflow as tf
from tensorflow.keras import  layers, models

print(tf.__version__)

In [0]:
#@title TCN
from typing import List

import tensorflow.keras.backend as K
import tensorflow.keras.layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Activation, Lambda, add
from tensorflow.keras.layers import Conv1D, SpatialDropout1D
from tensorflow.keras.layers import Dense, BatchNormalization,Input
from tensorflow.keras.models import Model


class ResidualBlock(Layer):

    def __init__(self,
                 dilation_rate,
                 nb_filters,
                 kernel_size,
                 padding,
                 activation='relu',
                 dropout_rate=0,
                 kernel_initializer='he_normal',
                 use_batch_norm=False,
                 last_block=True,
                 **kwargs):

        # type: (int, int, int, str, str, float, str, bool, dict) -> None
        """Defines the residual block for the WaveNet TCN

        Args:
            x: The previous layer in the model
            training: boolean indicating whether the layer should behave in training mode or in inference mode
            dilation_rate: The dilation power of 2 we are using for this residual block
            nb_filters: The number of convolutional filters to use in this block
            kernel_size: The size of the convolutional kernel
            padding: The padding used in the convolutional layers, 'same' or 'causal'.
            activation: The final activation used in o = Activation(x + F(x))
            dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
            kernel_initializer: Initializer for the kernel weights matrix (Conv1D).
            use_batch_norm: Whether to use batch normalization in the residual layers or not.
            kwargs: Any initializers for Layer class.
        """

        self.dilation_rate = dilation_rate
        self.nb_filters = nb_filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.activation = activation
        self.dropout_rate = dropout_rate
        self.use_batch_norm = use_batch_norm
        self.kernel_initializer = kernel_initializer
        self.last_block = last_block

        super(ResidualBlock, self).__init__(**kwargs)

    def _add_and_activate_layer(self, layer):
        """Helper function for building layer

        Args:
            layer: Appends layer to internal layer list and builds it based on the current output
                   shape of ResidualBlock. Updates current output shape.

        """
        self.residual_layers.append(layer)
        self.residual_layers[-1].build(self.res_output_shape)
        self.res_output_shape = self.residual_layers[-1].compute_output_shape(self.res_output_shape)

    def build(self, input_shape):

        with K.name_scope(self.name):  # name scope used to make sure weights get unique names
            self.residual_layers = list()
            self.res_output_shape = input_shape

            for k in range(2):
                name = 'conv1D_{}'.format(k)
                with K.name_scope(name):  # name scope used to make sure weights get unique names
                    self._add_and_activate_layer(Conv1D(filters=self.nb_filters,
                                                        kernel_size=self.kernel_size,
                                                        dilation_rate=self.dilation_rate,
                                                        padding=self.padding,
                                                        name=name,
                                                        kernel_initializer=self.kernel_initializer))

                if self.use_batch_norm:
                    # TODO should be WeightNorm here, but using batchNorm instead
                    self._add_and_activate_layer(BatchNormalization())

                self._add_and_activate_layer(Activation('relu'))
                self._add_and_activate_layer(SpatialDropout1D(rate=self.dropout_rate))

            if not self.last_block:
                # 1x1 conv to match the shapes (channel dimension).
                name = 'conv1D_{}'.format(k+1)
                with K.name_scope(name):
                    # make and build this layer separately because it directly uses input_shape
                    self.shape_match_conv = Conv1D(filters=self.nb_filters,
                                                   kernel_size=1,
                                                   padding='same',
                                                   name=name,
                                                   kernel_initializer=self.kernel_initializer)
      
            else:
                 self.shape_match_conv = Lambda(lambda x: x, name='identity')

            self.shape_match_conv.build(input_shape)
            self.res_output_shape = self.shape_match_conv.compute_output_shape(input_shape)

            self.final_activation = Activation(self.activation)
            self.final_activation.build(self.res_output_shape)  # probably isn't necessary

            # this is done to force keras to add the layers in the list to self._layers
            for layer in self.residual_layers:
                self.__setattr__(layer.name, layer)

            super(ResidualBlock, self).build(input_shape)  # done to make sure self.built is set True

    def call(self, inputs, training=None):
        """

        Returns: A tuple where the first element is the residual model tensor, and the second
                 is the skip connection tensor.
        """
        x = inputs
        for layer in self.residual_layers:
            if isinstance(layer, SpatialDropout1D):
                x = layer(x, training=training)
            else:
                x = layer(x)

        x2 = self.shape_match_conv(inputs)
        res_x = add([x2, x])
        return [self.final_activation(res_x), x]

    def compute_output_shape(self, input_shape):
        return [self.res_output_shape, self.res_output_shape]


def process_dilations(dilations):
    def is_power_of_two(num):
        return num != 0 and ((num & (num - 1)) == 0)

    if all([is_power_of_two(i) for i in dilations]):
        return dilations

    else:
        new_dilations = [2 ** i for i in dilations]
        return new_dilations


class TCN(Layer):
    """Creates a TCN layer.

        Input shape:
            A tensor of shape (batch_size, timesteps, input_dim).

        Args:
            nb_filters: The number of filters to use in the convolutional layers.
            kernel_size: The size of the kernel to use in each convolutional layer.
            dilations: The list of the dilations. Example is: [1, 2, 4, 8, 16, 32, 64].
            nb_stacks : The number of stacks of residual blocks to use.
            padding: The padding to use in the convolutional layers, 'causal' or 'same'.
            use_skip_connections: Boolean. If we want to add skip connections from input to each residual block.
            return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.
            activation: The activation used in the residual blocks o = Activation(x + F(x)).
            dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
            kernel_initializer: Initializer for the kernel weights matrix (Conv1D).
            use_batch_norm: Whether to use batch normalization in the residual layers or not.
            kwargs: Any other arguments for configuring parent class Layer. For example "name=str", Name of the model.
                    Use unique names when using multiple TCN.

        Returns:
            A TCN layer.
        """

    def __init__(self,
                 nb_filters=64,
                 kernel_size=2,
                 nb_stacks=1,
                 dilations=(1, 2, 4, 8, 16, 32),
                 padding='causal',
                 use_skip_connections=True,
                 dropout_rate=0.0,
                 return_sequences=False,
                 activation='linear',
                 kernel_initializer='he_normal',
                 use_batch_norm=False,
                 **kwargs):

        self.return_sequences = return_sequences
        self.dropout_rate = dropout_rate
        self.use_skip_connections = use_skip_connections
        self.dilations = dilations
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.nb_filters = nb_filters
        self.activation = activation
        self.padding = padding
        self.kernel_initializer = kernel_initializer
        self.use_batch_norm = use_batch_norm

        if padding != 'causal' and padding != 'same':
            raise ValueError("Only 'causal' or 'same' padding are compatible for this layer.")

        if not isinstance(nb_filters, int):
            print('An interface change occurred after the version 2.1.2.')
            print('Before: tcn.TCN(x, return_sequences=False, ...)')
            print('Now should be: tcn.TCN(return_sequences=False, ...)(x)')
            print('The alternative is to downgrade to 2.1.2 (pip install keras-tcn==2.1.2).')
            raise Exception()

        # initialize parent class
        super(TCN, self).__init__(**kwargs)

    def build(self, input_shape):
        self.main_conv1D = Conv1D(filters=self.nb_filters,
                                  kernel_size=1,
                                  padding=self.padding,
                                  kernel_initializer=self.kernel_initializer)
        self.main_conv1D.build(input_shape)

        # member to hold current output shape of the layer for building purposes
        self.build_output_shape = self.main_conv1D.compute_output_shape(input_shape)

        # list to hold all the member ResidualBlocks
        self.residual_blocks = list()
        total_num_blocks = self.nb_stacks * len(self.dilations)
        if not self.use_skip_connections:
            total_num_blocks += 1  # cheap way to do a false case for below

        for s in range(self.nb_stacks):
            for d in self.dilations:
                self.residual_blocks.append(ResidualBlock(dilation_rate=d,
                                                          nb_filters=self.nb_filters,
                                                          kernel_size=self.kernel_size,
                                                          padding=self.padding,
                                                          activation=self.activation,
                                                          dropout_rate=self.dropout_rate,
                                                          use_batch_norm=self.use_batch_norm,
                                                          kernel_initializer=self.kernel_initializer,
                                                          last_block = len(self.residual_blocks)+1 == total_num_blocks,
                                                          name='residual_block_{}'.format(len(self.residual_blocks))))
                # build newest residual block
                self.residual_blocks[-1].build(self.build_output_shape)
                self.build_output_shape = self.residual_blocks[-1].res_output_shape

        # this is done to force keras to add the layers in the list to self._layers
        for layer in self.residual_blocks:
            self.__setattr__(layer.name, layer)

        self.lambda_layer = Lambda(lambda tt: tt[:, -1, :])
        self.lambda_ouput_shape = self.lambda_layer.compute_output_shape(self.build_output_shape)

    def compute_output_shape(self, input_shape):
        """
        Overridden in case keras uses it somewhere... no idea. Just trying to avoid future errors.
        """
        if not self.built:
            self.build(input_shape)
        if not self.return_sequences:
            return self.lambda_ouput_shape
        else:
            return self.build_output_shape

    def call(self, inputs, training=None):
        x = inputs
        x = self.main_conv1D(x)
        skip_connections = list()
        for layer in self.residual_blocks:
            x, skip_out = layer(x, training=training)
            skip_connections.append(skip_out)

        if self.use_skip_connections:
            x = add(skip_connections)
        if not self.return_sequences:
            x = self.lambda_layer(x)
        return x

    def get_config(self):
        """
        Returns the config of a the layer. This is used for saving and loading from a model
        :return: python dictionary with specs to rebuild layer
        """
        config = super(TCN, self).get_config()
        config['nb_filters'] = self.nb_filters
        config['kernel_size'] = self.kernel_size
        config['nb_stacks'] = self.nb_stacks
        config['dilations'] = self.dilations
        config['padding'] = self.padding
        config['use_skip_connections'] = self.use_skip_connections
        config['dropout_rate'] = self.dropout_rate
        config['return_sequences'] = self.return_sequences
        config['activation'] = self.activation
        config['use_batch_norm'] = self.use_batch_norm
        config['kernel_initializer'] = self.kernel_initializer

        return config


def compiled_tcn(num_feat,  # type: int
                 num_classes,  # type: int
                 nb_filters,  # type: int
                 kernel_size,  # type: int
                 dilations,  # type: List[int]
                 nb_stacks,  # type: int
                 max_len,  # type: int
                 padding='causal',  # type: str
                 use_skip_connections=True,  # type: bool
                 return_sequences=True,
                 regression=False,  # type: bool
                 dropout_rate=0.05,  # type: float
                 name='tcn',  # type: str,
                 kernel_initializer='he_normal',  # type: str,
                 activation='linear',  # type:str,
                 opt='adam',
                 lr=0.002,
                 use_batch_norm=False):
    # type: (...) -> keras.Model
    """Creates a compiled TCN model for a given task (i.e. regression or classification).
    Classification uses a sparse categorical loss. Please input class ids and not one-hot encodings.

    Args:
        num_feat: The number of features of your input, i.e. the last dimension of: (batch_size, timesteps, input_dim).
        num_classes: The size of the final dense layer, how many classes we are predicting.
        nb_filters: The number of filters to use in the convolutional layers.
        kernel_size: The size of the kernel to use in each convolutional layer.
        dilations: The list of the dilations. Example is: [1, 2, 4, 8, 16, 32, 64].
        nb_stacks : The number of stacks of residual blocks to use.
        max_len: The maximum sequence length, use None if the sequence length is dynamic.
        padding: The padding to use in the convolutional layers.
        use_skip_connections: Boolean. If we want to add skip connections from input to each residual block.
        return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.
        regression: Whether the output should be continuous or discrete.
        dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
        activation: The activation used in the residual blocks o = Activation(x + F(x)).
        name: Name of the model. Useful when having multiple TCN.
        kernel_initializer: Initializer for the kernel weights matrix (Conv1D).
        opt: Optimizer name.
        lr: Learning rate.
        use_batch_norm: Whether to use batch normalization in the residual layers or not.
    Returns:
        A compiled keras TCN.
    """

    dilations = process_dilations(dilations)

    input_layer = Input(shape=(max_len, num_feat))

    x = TCN(nb_filters, kernel_size, nb_stacks, dilations, padding,
            use_skip_connections, dropout_rate, return_sequences,
            activation, kernel_initializer, use_batch_norm, name=name)(input_layer)

    print('x.shape=', x.shape)

    def get_opt():
        if opt == 'adam':
            return optimizers.Adam(lr=lr, clipnorm=1.)
        elif opt == 'rmsprop':
            return optimizers.RMSprop(lr=lr, clipnorm=1.)
        else:
            raise Exception('Only Adam and RMSProp are available here')

    if not regression:
        # classification
        x = Dense(num_classes)(x)
        x = Activation('softmax')(x)
        output_layer = x
        model = Model(input_layer, output_layer)

        # https://github.com/keras-team/keras/pull/11373
        # It's now in Keras@master but still not available with pip.
        # TODO remove later.
        def accuracy(y_true, y_pred):
            # reshape in case it's in shape (num_samples, 1) instead of (num_samples,)
            if K.ndim(y_true) == K.ndim(y_pred):
                y_true = K.squeeze(y_true, -1)
            # convert dense predictions to labels
            y_pred_labels = K.argmax(y_pred, axis=-1)
            y_pred_labels = K.cast(y_pred_labels, K.floatx())
            return K.cast(K.equal(y_true, y_pred_labels), K.floatx())

        model.compile(get_opt(), loss='sparse_categorical_crossentropy', metrics=[accuracy])
    else:
        # regression
        x = Dense(1)(x)
        x = Activation('linear')(x)
        output_layer = x
        model = Model(input_layer, output_layer)
        model.compile(get_opt(), loss='mean_squared_error')
    print('model.x = {}'.format(input_layer.shape))
    print('model.y = {}'.format(output_layer.shape))
    return model


In [0]:
model = compiled_tcn(return_sequences=False,
                         num_feat=1,
                         num_classes=CLASSES,
                         nb_filters=20,
                         kernel_size=6,
                         dilations=[2 ** i for i in range(9)],
                         nb_stacks=1,
                         max_len=SAMPLE_HEIGHT * SAMPLE_WIDTH,
                         use_skip_connections=True)
his=model.fit_generator(fit_generator(),steps_per_epoch=STEPS_PER_EPOCH, epochs=NUMBER_OF_EPOCHS,max_queue_size=50,class_weight=CLASS_WEIGHTS)